In [24]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "dew_point_2m", "rain", "snowfall", "surface_pressure", "cloud_cover", "et0_fao_evapotranspiration", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(9).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe['time'] = (hourly_dataframe['date'].dt.hour)
hourly_dataframe['month'] = (hourly_dataframe['date'].dt.month)
print(hourly_dataframe)
print(hourly_dataframe.dtypes)



Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                          date  temperature_2m  dew_point_2m  rain  snowfall  \
0    2022-12-31 22:00:00+00:00         14.5585        8.2585   0.0       0.0   
1    2022-12-31 23:00:00+00:00         14.5585        7.8085   0.0       0.0   
2    2023-01-01 00:00:00+00:00         15.3085        7.2585   0.0       0.0   
3    2023-01-01 01:00:00+00:00         15.7085        6.6085   0.0       0.0   
4    2023-01-01 02:00:00+00:00         15.7585        6.0585   0.0       0.0   
...                        ...             ...           ...   ...       ...   
8755 2023-12-31 17:00:00+00:00          6.3585        4.0585   0.1       0.0   
8756 2023-12-31 18:00:00+00:00          6.2585        3.8085   0.0       0.0   
8757 2023-12-31 19:00:00+00:00          6.3085        3.7085   0.1       0.0   
8758 2023-12-31 20:00:00+00:00          6.6585        4.3

In [25]:
hourly_dataframe = hourly_dataframe.iloc[:,  1:]
hourly_dataframe.shape

(8760, 12)

In [26]:
hourly_dataframe

,temperature_2m,dew_point_2m,rain,snowfall,surface_pressure,cloud_cover,et0_fao_evapotranspiration,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,time,month
0,14.5585,8.2585,0.0,0.0,1006.151794,30.000002,0.079528,26.826641,11.008500,0.372,22,12
1,14.5585,7.8085,0.0,0.0,1006.450439,30.000002,0.088108,28.312117,10.908501,0.371,23,12
2,15.3085,7.2585,0.0,0.0,1006.263000,38.400002,0.111398,30.532158,11.458500,0.371,0,1
3,15.7085,6.6085,0.0,0.0,1006.368958,86.400002,0.125796,30.699368,11.458500,0.370,1,1
4,15.7585,6.0585,0.0,0.0,1006.768066,52.200005,0.134292,31.259941,11.458500,0.369,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,6.3585,4.0585,0.1,0.0,998.752502,99.600006,0.006527,13.891522,5.458500,0.401,17,12
8756,6.2585,3.8085,0.0,0.0,998.949951,85.800003,0.011220,17.072504,5.358500,0.400,18,12
8757,6.3085,3.7085,0.1,0.0,999.249573,100.000000,0.012409,16.954432,5.308500,0.399,19,12
8758,6.6585,4.3085,0.0,0.0,999.255249,73.500008,0.009095,15.725037,5.408500,0.399,20,12


In [28]:
hourly_dataframe.to_csv('dane\prawdziwa_pogoda2023.csv', index=False)